In [1]:
import pandas as pd
import dwsemear

import postgresemear as sql  # mover os arquivos para a pasta principal

dwsemear.atualize_metadados()

# EXTRACAO E TRANSFORMACAO

In [11]:
#dados = pd.read_csv('BaseClientes-FEV2019.csv',sep=';', encoding='latin-1',dtype=str)
dados = pd.read_csv('BaseClientes-SET2019.csv',sep=';', encoding='latin-1',dtype=str)

In [12]:
dados = dados[dados.con_sta != 'C']
dados = dados.reset_index(drop=True)
dados = dwsemear.limpa_nome_das_colunas(dados)

In [13]:
dwsemear.pegue_colunas('formalizacao')

['codigo_contrato', 'codigo_produto', 'codigo_proposta']

In [14]:
formalizacao = dwsemear.pegue_tabela('formalizacao') 

In [15]:
# Verifique se os contratos ja foram adicionados
if len(formalizacao) != 0:
    formalizacao_contratos = set(formalizacao.codigo_contrato)
    dados.loc[:,'Con Contrato'] = dados.loc[:,'Con Contrato'].astype(int)
    novos_contratos = set(dados.loc[:,'Con Contrato'])
    contrato_aparece_nos_dois = list(formalizacao_contratos.intersection(novos_contratos))
    # Descarte contratos que ja foram carregados
    dados.index = dados.loc[:,'Con Contrato']
    dados = dados.drop(contrato_aparece_nos_dois)
    dados = dados.reset_index(drop=True)

# CARGA

In [18]:
registros = []
for i in dados.index:
    registros.append((dwsemear.para_int(dados.loc[i,'Con Contrato']),
                      dwsemear.para_int(dados.loc[i,'Produto']),
                      dwsemear.para_int(dados.loc[i,'Pedido Jretail'])))

    

In [19]:
registros[0]

(54791460, 3101, 6324203)

In [20]:
dwsemear.insira_tabela('formalizacao', registros)

In [5]:
# Resete a tabela
#dwsemear.delete_tabela('formalizacao')
#dwsemear.crie_tabela('formalizacao')